In [7]:
import pandas as pd
import numpy as np

In [323]:
agents = pd.read_csv('agentData.csv')
cemetry = pd.read_csv('cemetryData.csv')
backgroundSpending = pd.read_csv('backgroundSpending.csv')
inventory = pd.read_csv('inventoryinformation.csv')

In [317]:
simDays = 1

In [325]:
def Prob(age):
    prob = (0.000002 * age **2 - 0.000109 * age + 0.001155)
    return prob

def StatusChange(prob):
    randNum = np.random.uniform(low = 0.0, high = 1.0)
    if prob > randNum:
        status = 'Dead'
    elif (prob * 2) > (randNum ):
        status = 'Sick'
    else:
        status = 'Healthy'
    return status

In [326]:
for day in range(simDays):
    
    #increase date by one
    agents['currentDate'] = agents['currentDate'] + 1
    #determine every npc's new age
    agents['age'] = (agents['currentDate'] - agents['dob']) /360
    
    #modify agents status based on age
    prob = agents['age'].apply(Prob)
    prob = prob * agents['livingStatus'].apply(lambda x: 2 if x == 'Sick' else 1)
    agents['livingStatus'] = prob.apply(StatusChange)
    
    #determine impacts on population from deaths
    newDeaths = agents[agents['livingStatus'] == 'Dead'] 
    cemetry = cemetry.append(newDeaths, ignore_index = True)
    agents = agents[agents['livingStatus'] != 'Dead']
    
    #assign marriages, not currently done
    
    #assign pregancy
    prob = agents['age'].apply(lambda x: 1.0 / (abs(18 - x) + 2))
    prob = prob * (1 - agents['gender'])
    prob[agents['age'] < 15] = 0
    prob[agents['pregant'] > 0 ] = 0
    prob = prob.apply(lambda x: np.random.binomial(n=1, p = x))
    agents['pregant'] = agents['pregant'] + 60 * prob
       
    
    #create new agents, births
    newnpcIndex = max(max(agents['npcIndex']), max(cemetry['npcIndex']))
    births = agents[agents['pregant'] == 1]
    for i in range(len(births)):
        newnpcIndex += 1
        births['npcIndex'].iloc[i] = newnpcIndex
        
    
    #births['firstName'] = births['firstName'].apply(lambda x: something to assign random name)
    births['dob'] = births['currentDate']
    births['age'] = (births['currentDate'] - births['dob']) /360
    births['gender'] =  np.random.binomial(n=1, p = 0.5, size = len(births))
    births['pregant'].iloc[:] = 0
    births['gold'].iloc[:] = 0
    agents = agents.append(births, ignore_index = True) 
    agents['pregant'] = agents['pregant'].apply(lambda x: x-1 if x > 0 else 0)
    
    
    #determine purchases by agentsnew
    agents['base_spend'] = agents[['income', 'base_expense']].min(axis = 1)
    agents['luxury_spend'] = (agents['income'] - agents['base_spend']) * (1 - agents['saveRate'])
    agents['gold'] = agents['gold'] + (agents['income'] - agents['base_expense']) * agents['saveRate']
    current_period_spend = agents.groupby('background')['base_spend', 'luxury_spend'].apply(sum)
    current_period_spend.reset_index(level=0, inplace=True) 
    backgroundSpending = backgroundSpending.merge(current_period_spend,
                                             on = 'background',
                                             how = 'left')

    backgroundSpending['basicSpend'] = backgroundSpending['base_spend'] * backgroundSpending['basicSpend%']
    backgroundSpending['luxurySpend'] = backgroundSpending['luxury_spend'] * backgroundSpending['luxurySpend%']

    backgroundSpending = backgroundSpending.drop(labels = ['base_spend', 'luxury_spend'], axis = 1)
    
    
    #need something here to include in "vistors"
        #upfront estimates change in population and wealth for world, by city
            #can do this by random normal on population for each city, mean can be based on world event
            #each city can have some economic status, which translates to a random normal with different means/variance
                #developed, developing, recession, depression, collasping, 
        #a % of world population will travl, make this some uniform distribution 1% to 10%
        #there spilt among cities will be based on share of global wealth
    
    
    
    #determine corporate revenue based on agent purchases
    itemSpend = backgroundSpending.groupby('item')['basicSpend', 'luxurySpend'].apply(sum)
    itemSpend['TotalSpend'] = itemSpend['basicSpend'] + itemSpend['luxurySpend']
    itemSpend.reset_index(level=0, inplace=True)

    inventory['total_spend'] = itemSpend['TotalSpend']
    #inventory['available'] = inventory['available'] - inventory['total_spend'] / inventory['local_price']
    
    #pay agents based on corporate profits
    
    #determine new corporations formed, based on required cash
    
    #agent asset impacts
        #when they get enough money people buy homes
            #buying a home will result in shifting cash to an asset filed
                #homes will generate $$ based on value of the assets
            #After buying a home, a portion of luxury spending goes assets, driving higher income
                #assets decline on a five year balance
    
    #assign agents to employment to new and existing corporations
    


C:\Users\Matt\Anaconda2\lib\site-packages\ipykernel\__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Matt\Anaconda2\lib\site-packages\ipykernel\__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [327]:
inventory

,corpTypeID,soldBy,itemID,items,local_price,whole_sale_price,world_export_price,world_import_price,available,total_spend,Unnamed: 10,Unnamed: 11
0,1,Tavern,1,ale,0.20,999999,0,0.160,-1228.370000,245.674,NaN,NaN
1,1,Tavern,2,wine - common,0.20,999999,0,0.160,0.000000,0.000,NaN,NaN
2,1,Tavern,3,wine - fine,10.00,999999,0,8.000,-5.145000,51.450,NaN,NaN
3,2,Squalid Inn,4,inn stay - squalid,0.07,999999,0,0.056,0.000000,0.000,NaN,NaN
4,3,Poor Inn,5,inn stay - poor,0.10,999999,0,0.080,-524.800000,52.480,NaN,NaN
5,4,Modest Inn,6,inn stay - modest,0.50,999999,0,0.400,0.000000,0.000,NaN,NaN
6,5,Comfortable Inn,7,inn stay - comfortable,0.80,999999,0,0.640,0.000000,0.000,NaN,NaN
7,6,Wealthy Inn,8,inn stay - wealthy,2.00,999999,0,1.600,-94.852000,189.704,NaN,;
8,7,Arisocratic Inn,9,inn stay aristocratic,4.00,999999,0,3.200,0.000000,0.000,NaN,NaN
9,2,Squalid Inn,10,meal - squalid,0.03,999999,0,0.024,-1029.000000,30.870,NaN,NaN
